# Spurs Data Analysis

### I loaded the necessary packages

In [212]:
import pandas as pd
import numpy as np
import csv
import os

### I imported all the csvs from the <b>"raw_season_csvs"</b> folder and combined them into one dataframe
It appears to all be correct (each season should have 380 total games, so one game per row results in 380 rows per file).

<br><i>The data comes from [football-data.co.uk](https://www.football-data.co.uk/englandm.php)</i>
<br><i>The below cell was written with the help of Claude Code</i>

In [213]:
csv_dir = "/Users/hgorledeenn/Desktop/CJS/0126data_studio/project-1-spurs/raw_season_csvs"

csv_files = sorted([f for f in os.listdir(csv_dir) if f.endswith('.csv')])
all_rows = []
all_columns = set()

for file in csv_files:
    filepath = os.path.join(csv_dir, file)
    row_count = 0
    with open(filepath, 'r', encoding='latin-1') as f:
        reader = csv.DictReader(f)
        for row in reader:
            row['season'] = file.split('.')[0].split('-')[1]
            all_rows.append(row)
            all_columns.update(row.keys())
            row_count += 1
    print(f"Loaded {file}: {row_count} rows")

combined_df = pd.DataFrame(all_rows)

print(f"\nCombined dataframe shape: {combined_df.shape}")
print(f"Number of columns: {len(combined_df.columns)}")


Loaded 10-0203.csv: 380 rows
Loaded 11-0304.csv: 380 rows
Loaded 12-0405.csv: 380 rows
Loaded 13-0506.csv: 380 rows
Loaded 14-0607.csv: 380 rows
Loaded 15-0708.csv: 380 rows
Loaded 16-0809.csv: 380 rows
Loaded 17-0910.csv: 380 rows
Loaded 18-1011.csv: 380 rows
Loaded 19-1112.csv: 380 rows
Loaded 20-1213.csv: 380 rows
Loaded 21-1314.csv: 380 rows
Loaded 22-1415.csv: 381 rows
Loaded 23-1516.csv: 380 rows
Loaded 24-1617.csv: 380 rows
Loaded 25-1718.csv: 380 rows
Loaded 26-1819.csv: 380 rows
Loaded 27-1920.csv: 380 rows
Loaded 28-2021.csv: 380 rows
Loaded 29-2122.csv: 380 rows
Loaded 30-2223.csv: 380 rows
Loaded 31-2324.csv: 380 rows
Loaded 32-2425.csv: 380 rows
Loaded 33-2526.csv: 260 rows
Loaded 8-0001.csv: 380 rows
Loaded 9-0102.csv: 380 rows

Combined dataframe shape: (9761, 218)
Number of columns: 218


### I selected only the columns I need for my analysis and saved the pared down dataframe as a csv

In [214]:
all_games = combined_df[['season','Date', 'HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST']]

In [215]:
all_games.to_csv('all_games.csv', index=False)

In [216]:
all_games.head()

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST
0,0203,17/08/2002,Blackburn,Sunderland,0,0,D,0,0,D,15,7,5,3
1,0203,17/08/2002,Charlton,Chelsea,2,3,A,2,1,H,5,21,5,12
2,0203,17/08/2002,Everton,Tottenham,2,2,D,1,0,H,13,10,9,5
3,0203,17/08/2002,Fulham,Bolton,4,1,H,3,1,H,13,3,6,1
4,0203,17/08/2002,Leeds,Man City,3,0,H,2,0,H,13,18,8,10


### Filter to only games where Tottenham is the home or away team, reset the index, standardize the date column

In [217]:
only_tot = all_games[(all_games['HomeTeam'] == 'Tottenham') | (all_games['AwayTeam'] == 'Tottenham')]
only_tot = only_tot.reset_index(drop=True)
only_tot['Date'] = pd.to_datetime(only_tot['Date'], dayfirst=True, format='mixed')
only_tot.head()

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST
0,0203,2002-08-17,Everton,Tottenham,2,2,D,1,0,H,13,10,9,5
1,0203,2002-08-24,Tottenham,Aston Villa,1,0,H,1,0,H,8,18,3,8
2,0203,2002-08-27,Charlton,Tottenham,0,1,A,0,1,A,10,7,5,1
3,0203,2002-08-31,Tottenham,Southampton,2,1,H,1,1,D,18,15,8,8
4,0203,2002-09-11,Fulham,Tottenham,3,2,H,0,2,A,13,7,8,4


### Make a blank dataframe for my tottenham-centric stats

In [218]:
tottenham = pd.DataFrame()

### Defining a few functions for use below

In [219]:
def ft_results(row):
    if row['TFTG'] > row['OTFTG']:
        return 'W'
    elif row['TFTG'] < row['OTFTG']:
        return 'L'
    else:
        return 'D'
    
def ht_results(row):
    if row['THTG'] > row['OTHTG']:
        return 'W'
    elif row['THTG'] < row['OTHTG']:
        return 'L'
    else:
        return 'D'

### Data Diary for my columns:
<b>Season</b>: Season of the game (in the format 0203, where the season spanned 2002 and 2003)
<br><b>Date</b>: Date of the game
<br><b>OT</b>: The team Tottenham played
<br><b>TL</b>: The location of the game, from Tottenham's perspective (H = Home, A = Away)
<br><b>TFTG</b>: Tottenham Full-Time Goals
<br><b>OTFTG</b>: Other Team Full-Time Goals
<br><b>FTR</b>: Full Time Result (W if Tottenham win, L if Tottenham loss, D if draw)
<br><b>THTG</b>: Tottenham Half-Time Goals
<br><b>OTHTG</b>: Other Team Half-Time Goals
<br><b>HTR</b>: Half Time Result (W if Tottenham winning, L if Tottenham losing, D if tied)
<br><b>TS</b>: Total shots by Tottenham
<br><b>OTS</b>: Total shots by the Other Team
<br><b>TST</b>: Tottenham shots on target
<br><b>OTST</b>: Other Team shots on target
<br><b>FTGD</b>: Full-Time goal differential (+ if Tottenham won, - if Tottenham lost, 0 if draw)
<br><b>HTGD</b>: Half-Time goal differential (+ if Tottenham won, - if Tottenham lost, 0 if draw)


In [220]:
tottenham['Season'] = only_tot['season']
tottenham['Date'] = only_tot['Date']
tottenham['OT'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AwayTeam'], only_tot['HomeTeam'])
tottenham['TL'] = np.where(only_tot['HomeTeam'] == 'Tottenham', "H", "A")
tottenham['TFTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['FTHG'], only_tot['FTAG'])
tottenham['OTFTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['FTAG'], only_tot['FTHG'])
tottenham['FTR'] = tottenham.apply(ft_results, axis=1)
tottenham['THTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HTHG'], only_tot['HTAG'])
tottenham['OTHTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HTAG'], only_tot['HTHG'])
tottenham['HTR'] = tottenham.apply(ht_results, axis=1)
tottenham['TS'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HS'], only_tot['AS'])
tottenham['OTS'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AS'], only_tot['HS'])
tottenham['TST'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HST'], only_tot['AST'])
tottenham['OTST'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AST'], only_tot['HST'])


In [221]:
numeric_columns = ['TFTG', 'OTFTG', 'THTG', 'OTHTG', 'TS', 'OTS', 'TST', 'OTST']

for col in numeric_columns:
    tottenham[col] = pd.to_numeric(tottenham[col], errors='coerce')

tottenham.dtypes

Season            object
Date      datetime64[ns]
OT                object
TL                object
TFTG               int64
OTFTG              int64
FTR               object
THTG               int64
OTHTG              int64
HTR               object
TS                 int64
OTS                int64
TST                int64
OTST               int64
dtype: object

In [222]:
tottenham['FTGD'] = tottenham['TFTG'] - tottenham['OTFTG']
tottenham['HTGD'] = tottenham['THTG'] - tottenham['OTHTG']


In [223]:
tottenham.head()

,Season,Date,OT,TL,TFTG,OTFTG,FTR,THTG,OTHTG,HTR,TS,OTS,TST,OTST,FTGD,HTGD
0,0203,2002-08-17,Everton,A,2,2,D,0,1,L,10,13,5,9,0,-1
1,0203,2002-08-24,Aston Villa,H,1,0,W,1,0,W,8,18,3,8,1,1
2,0203,2002-08-27,Charlton,A,1,0,W,1,0,W,7,10,1,5,1,1
3,0203,2002-08-31,Southampton,H,2,1,W,1,1,D,18,15,8,8,1,0
4,0203,2002-09-11,Fulham,A,2,3,L,2,0,W,7,13,4,8,-1,2


In [224]:
tottenham.to_csv('tottenham_centric.csv', index=False)